In [1]:
import numpy as np
import pandas as pd
import urllib.request
from ultralytics import YOLO
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import FastText
from keras.models import load_model
import fasttext
from khaiii import KhaiiiApi

ModuleNotFoundError: No module named 'khaiii'

In [ ]:
api = KhaiiiApi()

def seperate(sentence):
    sentence = str(sentence)

    result = []

    for word in api.analyze(sentence):
        for morph in word.morphs:
            result.append(morph.lex)

    return ' '.join(result)

In [ ]:
import cv2
import glob
import time
import math
import json
from ultralytics import YOLO
import matplotlib.pyplot as plt
from easyocr import Reader
import numpy as np
from PIL import ImageFont, ImageDraw, Image

# 검출율, 인식율 계산을 위한 json파일
with open('./answer.json', 'r', encoding='UTF-8') as f:
    answer_data = json.load(f)

# 사전에 학습된 YOLOv8모델 호출
print('YOLOv8-segmentation 모델 호출 중...')
model = YOLO('../runs/segment/train10/weights/best.pt')

# EasyOCR 모델 호출
print('EasyOCR모델 호출중...')
reader = Reader(lang_list=['en', 'ko'])

# 사전에 학습된 카테고리 분류용 fasttext모델 호출
print('카테고리 분류용 모델 호출중...')
text_model = fasttext.load_model('./receipts_fasttext_240723')

#대상 이미지 경로
images = glob.glob('../img/*')
print(f'{len(images)}장의 이미지 불러오는 중...')
# img = '../img/IMG_2016.jpg'

count = 0
time_list = []
detect_points_list = []
recognize_points_list = []

for img in images:
    filename = img.split('/')[-1]

    # 정답용 빈 리스트
    vet_name = ''
    vet_address = ''
    treat_name = []
    price = []
    date = ''
    
    for data in answer_data:
        if data['파일명'] == filename:
            vet_name = data['동물병원 이름']
            vet_address = data['동물병원 주소']
            date = data['발행일']
            
            for treat in data['진료내역']:
                treat_name.append(treat['진료명'])
                price.append(treat['금액'])

    # 이미지에서 글자추출에 얼마나 걸리는지 계산용 시작시간 기록
    start = time.time()
    print('시작시간 기록:', start)
    
    # YOLOv8모델로 텍스트영역 골라내기
    # print(f'YOLOv8 모델로 {count}번 이미지에서 필요한 텍스트영역 추출중...')
    print(f'YOLOv8 모델로 이미지{filename}에서 필요한 텍스트영역 추출중...')
    results = model.predict(img)
    image= cv2.imread(img)

    # 정리용 빈 리스트
    vet_name_list = ''
    vet_address_list = ''
    treat_name_list = []
    price_list = []
    date_list = ''

    print('검출된 이미지에 EasyOCR모델 적용중...')
    for i, box in enumerate(results[0].boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding Box 좌표 추출
        cropped_img = image[y1:y2, x1:x2]  # 이미지 크롭
        
        # 1. 그레이스케일 변환
        gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

        # img_result = reader.readtext(adjusted, batch_size=10000, detail=0)
        img_result = reader.readtext(gray, batch_size=10000, detail=0, paragraph=True, x_ths = 1.19, y_ths = 0.34, slope_ths=0.5, width_ths=0.99, height_ths=0.6)
        # img_result = reader.readtext(gray, batch_size=10000, detail=0, paragraph=True, y_ths = -0.001)
        print('검출 결과:',img_result)

        for result in img_result:
            category = text_model.predict(seperate(result))[0][0]
            if ('발' in result) and ('행' in result) and ('년' in result):
                text = result.replace('.', ':').replace(',', ':').replace(';', ':').split(' ')
                if len(text[-4].strip()) == 2:
                    text[-4] = '0'+text[-4].replace('월', '').strip()
                else:
                    text[-4] = text[-4].replace('월', '').strip()

                if len(text[-3]) == 2:
                    text[-3] = '0'+text[-3].replace('일', '').strip()
                else:
                    text[-3] = text[-3].replace('일', '').strip()
                
                text[-5] = text[-5].replace('년', '').strip()

                date_str = '-'.join(text[-5:-2])
                # datetime = list(map(int, text[-1].split(':')))
                datetime = text[-1].split(':')
                if text[-2] == '오후':
                    datetime[0] = int(datetime[0])
                    datetime[0] += 12
                    datetime[0] = str(datetime[0])
                datetime = list(map(str, datetime))
                time_str = ':'.join(datetime)
                date_list = ' '.join([date_str, time_str])
                break
            
            elif 'price' in category:
                price_list.append(result)

            elif (len(result) > 1) and (('사업자' not in result) and (':' not in result)):
                treat_name_list.append(result)
            # elif result.find('소재지') > 0:
            elif len(result) > 1 and(' 전 'in result):
                text = result.split('명 ')[1].split(' 전 ')[0]
                vet_name_list = text.split(' 원 ')[0].replace(':', ' ').strip()
                vet_address_list = text.split('소재지')[1].replace(':', ' ').strip()
                break
                
    end = time.time()
    print('종료시간 기록 :', end)
    time_list.append(end-start)
    print('소요시간:', f'{end - start} sec')

    # print('====================================================')

    points_detect = []
    points_recognize = []
    print(' ')
    print('결과물 모두 출력 >> ')

    #=================================================================================

    print('검출된 동물병원 이름 :',vet_name_list, "/ 정답 동물병원 이름 :", vet_name)
    points_detect.append(len(vet_name_list.replace(' ', '')) / len(vet_name.replace(' ', '')) * 100)
    print('동물병원 이름 검출율:', points_detect[0])
    vet_name_point = 0
    for i in range(min(len(vet_name_list.replace(' ', '')), len(vet_name.replace(' ', '')))):
        if vet_name.replace(' ', '')[i] == vet_name_list.replace(' ', '')[i]:
            vet_name_point += 1
    points_recognize.append(vet_name_point / len(vet_name.replace(' ', '')) * 100)
    print('동물병원 이름 인식율:', points_recognize[0])

    #=================================================================================

    print('검출된 동물병원 주소 :',vet_address_list, "/ 정답 동물병원 주소 :", vet_address)
    points_detect.append(len(vet_address_list.replace(' ', '')) / len(vet_address.replace(' ', '')) * 100)
    print('동물병원 주소 검출율:', points_detect[1])
    vet_address_point = 0
    for i in range(min(len(vet_address.replace(' ', '')), len(vet_address_list.replace(' ', '')))):
        if vet_address.replace(' ', '')[i] == vet_address_list.replace(' ', '')[i]:
            vet_address_point += 1
    points_recognize.append(vet_address_point / len(vet_address.replace(' ', '')) * 100)
    print('동물병원 주소 인식율:', points_recognize[1])

    #=================================================================================

    print('진료명 :',treat_name_list)
    print('진료답 :',treat_name)
    treat_name_detect = []
    treat_name_recognize = []
    for i in range(min(len(treat_name), len(treat_name_list))):
        treat_name_point = 0
        treat_name_detect.append(len(treat_name_list[i].replace(' ', '')) / len(treat_name[i].replace(' ', '')) * 100)
        for j in range(min(len(treat_name[i].replace(' ', '')), len(treat_name_list[i].replace(' ', '')))):
            if treat_name[i].replace(' ', '')[j] == treat_name_list[i].replace(' ', '')[j]:
                treat_name_point += 1
        treat_name_recognize.append(treat_name_point / len(treat_name[i].replace(' ', '')) * 100)
    
    points_detect.append(sum(treat_name_detect) / len(treat_name_detect))
    points_recognize.append(sum(treat_name_recognize) / len(treat_name_recognize))
    print('진료명 검출율 :', points_detect[2])
    print('진료명 인식율 :', points_recognize[2])

    #=================================================================================

    print('진료비 :',price_list)
    print('비용답 :',price)
    price_detect = []
    price_recognize = []
    for i in range(min(len(price), len(price_list))):
        price_point = 0
        price_detect.append(len(price_list[i].replace(' ', '')) / len(price[i].replace(' ', '')) * 100)
        for j in range(min(len(price[i]), len(price_list[i]))):
            if price[i].replace(' ', '')[j] == price_list[i].replace(' ', '')[j]:
                price_point += 1
        price_recognize.append(price_point / len(price[i].replace(' ', '')) * 100)
    
    points_detect.append(sum(price_detect) / len(price_detect))
    points_recognize.append(sum(price_recognize) / len(price_recognize))
    print('진료비 검출율 :', points_detect[3])
    print('진료비 인식율 :', points_recognize[3])

    #=================================================================================

    print('진료일 :',date_list, '/ 일자답 :', date)
    points_detect.append(len(date_list.replace(' ', '')) / len(date.replace(' ', '')) * 100)
    print('진료일 검출율:', points_detect[4])
    date_point = 0
    for i in range(min(len(date_list.replace(' ', '')), len(date.replace(' ', '')))):
        if date.replace(' ', '')[i] == date_list.replace(' ', '')[i]:
            date_point += 1
    points_recognize.append(date_point / len(date.replace(' ', '')) * 100)
    print('진료일 인식율:', points_recognize[4])

    # print('====================================================')
    print(' ')

    print(f'{filename} 이미지의 평균 검출율 :', sum(points_detect) / len(points_detect))
    detect_points_list.append(sum(points_detect) / len(points_detect))
    print(f'{filename} 이미지의 평균 인식율 :', sum(points_recognize) / len(points_recognize))
    recognize_points_list.append(sum(points_recognize) / len(points_recognize))

    count += 1
    print('====================================================')
    print(' ')

print(f'총 {len(images)}개의 영수증 이미지 평균 검출 시간 : {sum(time_list)/len(time_list)} sec')
print(f'총 {len(images)}개의 영수증 이미지 평균 검출율 : {sum(detect_points_list)/len(detect_points_list)}%')
print(f'총 {len(images)}개의 영수증 이미지 평균 인식율 : {sum(recognize_points_list)/len(recognize_points_list)}%')
# print(f'총 {len(images)}개의 영수증 이미지 평균 검출 시간 : {(end - start) / len(images)} sec')


YOLOv8-segmentation 모델 호출 중...
EasyOCR모델 호출중...
카테고리 분류용 모델 호출중...
50장의 이미지 불러오는 중...
시작시간 기록: 1722409753.596303
YOLOv8 모델로 이미지IMG_2039.jpg에서 필요한 텍스트영역 추출중...

image 1/1 /Users/heonyounglee/Desktop/ocr/segmentation/../img/IMG_2039.jpg: 640x416 1 date, 2 text_areas, 157.8ms
Speed: 10.8ms preprocess, 157.8ms inference, 14.5ms postprocess per image at shape (1, 3, 640, 416)
검출된 이미지에 EasyOCR모델 적용중...
검출 결과: ['사업자 등록번호 301-09-95188 병 원 명 : 정성환동물병원 원 장 : 정성환 사업자 소재지 : 충청북도 청주시 청원구 공항로 5 전  화 번 호 : 043-216-8274']
검출 결과: ['진료비 초진', '11,000', '검사 내과(혈액) 전해질   분석 (ABL8O), GAS 검사 내과(혈액) CRP 검사   내과 체장영 (cPLI)   정량검 사 X-ray(방사선) / 흥복부 (~1Okg)', '45,000', '38,000', '77,000', '70,000']
검출 결과: ['발 행 일 2023년 10월 15일 오전 11:49:59']
종료시간 기록 : 1722409769.116309
소요시간: 15.520005941390991 sec
 
결과물 모두 출력 >> 
검출된 동물병원 이름 : 정성환동물병원 / 정답 동물병원 이름 : 정성환동물병원
동물병원 이름 검출율: 100.0
동물병원 이름 인식율: 100.0
검출된 동물병원 주소 : 충청북도 청주시 청원구 공항로 5 / 정답 동물병원 주소 : 충청북도 청주시 청원구 공항로 5
동물병원 주소 검출율: 100.0
동물병원 주소 인식율: 100.0
진료명 : ['진료비 초진'

In [ ]:
import json

with open('./answer.json', 'r', encoding='UTF-8') as f:
    answer_data = json.load(f)

answer_data[0]

{'파일명': 'IMG_2048.jpg',
 '동물병원 이름': '서울동물의료센터',
 '동물병원 주소': '세종시 아름동 1089 (보듬7로 78), 2층',
 '발행일': '2023-01-16 14:00:44',
 '진료내역': [{'진료명': '내복약 BID-일반(~5kg)', '금액': '4,400'},
  {'진료명': '내복약 BID-일반(~5kg) 경련약', '금액': '4,400'},
  {'진료명': '시그니처바이 반려묘 A/a x5개 스틱', '금액': '6,000'},
  {'진료명': '시그니처바이 A/a x8개 캔', '금액': '11,000'}]}